In [ ]:
#| default_exp visualization

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#Weight & Biases
import wandb

#Yaml
from yaml import load, FullLoader

#Embeddings
from dvats.all import *
from tsai.data.preparation import prepare_forecasting_data
from tsai.data.validation import get_forecasting_splits
from fastcore.all import *

#Dimensionality reduction
from tsai.imports import *

#Clustering
import hdbscan
import utils.config as cfg_
import time

### Utilities for simpler code in 05_xai-lrp

In [2]:
#| export
def get_dataset(config, run_dr, config_dr, print_flag):
    # Botch to use artifacts offline
    artifacts_gettr = run_dr.use_artifact if config_dr.use_wandb else api.artifact
    enc_artifact = artifacts_gettr(config.enc_artifact, type='learner')
    if print_flag: print (enc_artifact.name)
    ## TODO: This only works when you run it two timeS! WTF?
    try:
        enc_learner = enc_artifact.to_obj()
    except:
        enc_learner = enc_artifact.to_obj()
    #enc_artifact.metadata
    ## Restore artifact
    enc_logger = enc_artifact.logged_by()
    enc_artifact_train = artifacts_gettr(enc_logger.config['train_artifact'], type='dataset')
    if enc_logger.config['valid_artifact'] is not None:
        enc_artifact_valid = artifacts_gettr(enc_logger.config['valid_artifact'], type='dataset')
        if print_flag: print("enc_artifact_valid:", enc_artifact_valid.name)
    if print_flag: print("enc_artifact_train: ", enc_artifact_train.name)
    if config_dr.dr_artifact is not None:
        dr_artifact = artifacts_gettr(config_dr.dr_artifact)
    else:
        dr_artifact = enc_artifact_train
    if print_flag: print("DR artifact train: ", dr_artifact.name)
    df = dr_artifact.to_df()
    if print_flag: display(df.head())
    return df, dr_artifact, enc_artifact, enc_learner